In [2]:
%pip install git+https://github.com/yfukai/BaSiCPy@autosegment

  Cloning https://github.com/yfukai/BaSiCPy (to revision autosegment) to /tmp/pip-req-build-i9waw88g
  Running command git clone --filter=blob:none --quiet https://github.com/yfukai/BaSiCPy /tmp/pip-req-build-i9waw88g
  Running command git checkout -b autosegment --track origin/autosegment
  Switched to a new branch 'autosegment'
  Branch 'autosegment' set up to track remote branch 'autosegment' from 'origin'.
  Resolved https://github.com/yfukai/BaSiCPy to commit 2ebf6939a94919fca8271185e38d0600fb7c1140
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for BaSiCPy: filename=BaSiCPy-1.1.0-py3-none-any.whl size=24603 sha256=14b561303fb9d70d3b667e58ae0dc70a3a49ebe8b0f29febcfa339c278965488
  Stored in directory: /tmp/pip-ephem-wheel-cache-7lkhhr35/wheels/01/7f/b0/c1f8ae9c14413f3fb55a0d9c82e7868e7bf31b4de043e9ab8f
Successfully built BaSiCPy
  Attempting uninstall: BaSiCPy
    Found existing

# Install and import 

In [6]:
workdir = "/work/fukai/basicpy"
os.makedirs(workdir,exist_ok = True)

In [3]:
import numpy as np
import pandas as pd
from basicpy import BaSiC, datasets, metrics
from itertools import product
from tqdm import tqdm
from matplotlib import pyplot as plt
import pickle 
from os import path
from skimage import filters

plt.rcParams["font.family"] = ""

In [ ]:
with open(path.join(workdir,"wo_darkfield_imagess.pickle"),"rb") as f:
    imagess=pickle.load(f)

In [ ]:
parameters_df=pd.read_csv(path.join(workdir,"wo_darkfield_parameters.csv"))

In [ ]:
flatfields = []
baselines = []
b = BaSiC(
    get_darkfield=False,
)
estimated_parameters = []
for p,images in tqdm(zip(parameters_df.to_dict(orient="records"),imagess),total=len(imagess)):
    for fitting_mode, smoothness_flatfield in product(
        ["ladmap", "approximate"],  # fitting_mode
        list(np.logspace(-3, 1, 7)),  # smoothness_flatfield
    ):
        p2 = p.copy()
        p2.update({
            "fitting_mode": fitting_mode,
            "smoothness_flatfield": smoothness_flatfield,
        })
        b.fitting_mode=fitting_mode
        b.smoothness_flatfield=smoothness_flatfield
        b.fit(images)
        flatfields.append(b.flatfield)
        baselines.append(b.baseline)
        estimated_parameters.append(p2)
estimated_parameters_df = pd.DataFrame.from_records(estimated_parameters).reset_index(drop=True)

In [ ]:
np.save("flatfields.npy",flatfields)

# Calculate deviation

In [29]:
flatfields = np.load("flatfields.npy")

In [35]:
assert len(estimated_parameters_df) == len(flatfields)

In [36]:
for (i, row), flatfield in zip(estimated_parameters_df.iterrows(),flatfields):
    estimated_parameters_df.loc[i,"deviation"] = np.mean(np.abs(flatfield - flatfield_profiles[row["true_flatfield_id"]]))

In [37]:
if "Unnamed: 0" in estimated_parameters_df.keys():
    del estimated_parameters_df["Unnamed: 0"]

In [38]:
estimated_parameters_df.to_csv("wo_flatfield_estimated_parameters_df.csv")
estimated_parameters_df.head()

,true_flatfield_id,intensity,ave_count,n_images,blob_dist_key,rep,fitting_mode,smoothness_flatfield,deviation
0,0,10,10,10,biased,0,ladmap,0.001000,0.036881
1,0,10,10,10,biased,0,ladmap,0.004642,0.036452
2,0,10,10,10,biased,0,ladmap,0.021544,0.033683
3,0,10,10,10,biased,0,ladmap,0.100000,0.029211
4,0,10,10,10,biased,0,ladmap,0.464159,0.018625


In [ ]:
estimated_parameters_df = pd.read_csv("wo_flatfield_estimated_parameters_df.csv")

In [39]:
estimated_parameters_df.keys()

Index(['true_flatfield_id', 'intensity', 'ave_count', 'n_images',
       'blob_dist_key', 'rep', 'fitting_mode', 'smoothness_flatfield',
       'deviation'],
      dtype='object')

In [40]:
res_dir = "figs/wo_darkfields/"
os.makedirs(res_dir,exist_ok=True)

# Plot profiles

In [ ]:
for (intensity, ave_count, n_images, blob_dist_key), grp in tqdm(estimated_parameters_df.groupby(["intensity", "ave_count", "n_images","blob_dist_key"])):
    df = grp[grp["rep"]==0]
    smoothness_flatfields=df["smoothness_flatfield"].unique()
    ncol = len(smoothness_flatfields)
    nrow = int(len(df) / ncol)
    fig, axes = plt.subplots(nrow,ncol, figsize=(2*ncol,2*nrow))
    for irow, ((fitting_mode, true_flatfield_id), grp2) in enumerate(df.groupby(["fitting_mode","true_flatfield_id"])):
        df2=grp2.sort_values(["smoothness_flatfield",])
        assert len(axes[irow,:]) == len(df2)
        for ax, (j, row) in zip(axes[irow,:],df2.iterrows()):
            ax.imshow(flatfields[j])
            #ax.axis("off")
        axes[irow,0].set_ylabel(f"{fitting_mode}\nflatfield #{true_flatfield_id}",rotation=90)
    fig.tight_layout()
    fig.savefig(os.path.join(res_dir,f"intensity_{intensity}_ave-count_{ave_count}_n-images{n_images}_blob-dist_{blob_dist_key}.png"),bbox_inches="tight")